# **AFRICAN INSTITUTE FOR MATHEMATICAL SCIENCE - RWANDA**

---
# **Big Data Analytics with Python**
## Assignment 1
## Name: Latifah AKIMANA
### <span style="color:red">Note: all datasets where shared in the classroom</span>
---

# **Part 1: Reading, Writing and Validating Data in PySpark HW**

Welcome to your first coding homework assignment in PySpark! I hope you enjoyed the lecture on Reading, Writing and Validating dataframes. Now it's time to put what you've learned into action! 

I've included several instructions below to help guide you through this homework assignment which I hope will get you feeling even comfortable reading, writing and validating dataframes. If you get stuck at any point, feel free to jump to the next lecture where I will guide you through my solutions to the HW assignment. 

Have fun!

Let's dig right in!


## But first things first.....
We need to always begin every Spark session by creating a Spark instance. Let's go ahead and use the method we learned in the lecture in the cell below. Also see if you can remember how to open the Spark UI (using a link that automatically guides you there). 

In [108]:
%pip install findspark

Note: you may need to restart the kernel to use updated packages.


In [109]:
%pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Assignment1").getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/18 15:53:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/18 15:53:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Next let's start by reading a basic csv dataset

Download the pga_tour_historical dataset that is attached to this lecture and save it whatever folder you want, then read it in. 

**Data Source:** https://www.kaggle.com/bradklassen/pga-tour-20102018-data

Rememer to try letting Spark infer the header and infer the Schema types!

In [3]:
historical_dataframe = spark.read.csv('Datasets/pga_tour_historical.csv', inferSchema=True, header=True)

## 1. View first 5 lines of dataframe
First generate a view of the first 5 lines of the dataframe to get an idea of what is inside. We went over two ways of doing this... see if you can remember BOTH ways. 

In [4]:
historical_dataframe.show(5)

+---------------+------+----------------+--------------------+-----+
|    Player Name|Season|       Statistic|            Variable|Value|
+---------------+------+----------------+--------------------+-----+
|Robert Garrigus|  2010|Driving Distance|Driving Distance ...|   71|
|   Bubba Watson|  2010|Driving Distance|Driving Distance ...|   77|
| Dustin Johnson|  2010|Driving Distance|Driving Distance ...|   83|
|Brett Wetterich|  2010|Driving Distance|Driving Distance ...|   54|
|    J.B. Holmes|  2010|Driving Distance|Driving Distance ...|  100|
+---------------+------+----------------+--------------------+-----+
only showing top 5 rows


In [5]:
# another way
historical_dataframe.limit(5).toPandas()

,Player Name,Season,Statistic,Variable,Value
0,Robert Garrigus,2010,Driving Distance,Driving Distance - (ROUNDS),71
1,Bubba Watson,2010,Driving Distance,Driving Distance - (ROUNDS),77
2,Dustin Johnson,2010,Driving Distance,Driving Distance - (ROUNDS),83
3,Brett Wetterich,2010,Driving Distance,Driving Distance - (ROUNDS),54
4,J.B. Holmes,2010,Driving Distance,Driving Distance - (ROUNDS),100


## 2. Print the schema details

Now print the details of the dataframes schema that Spark infered to ensure that it was infered correctly. Sometimes it is not infered correctly, so we need to watch out!

In [6]:
historical_dataframe.printSchema()

root
 |-- Player Name: string (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Statistic: string (nullable = true)
 |-- Variable: string (nullable = true)
 |-- Value: string (nullable = true)



## 3. Edit the schema during the read in

We can see from the output above that Spark did not correctly infer that the "value" column was an integer value. Let's try specifying the schema this time to let spark know what the schema should be.

Here is a link to see a list of PySpark data types in case you need it (also attached to the lecture): 
https://spark.apache.org/docs/latest/sql-ref-datatypes.html

In [7]:
historical_dataframe.schema['Value'].dataType

StringType()

## 4. Generate summary statistics for only one variable

See if you can generate summary statistics for only the "Value" column using the .describe function

(count, mean, stddev, min, max) 

In [9]:
historical_dataframe.describe(['Value']).show()

+-------+------------------+
|summary|             Value|
+-------+------------------+
|  count|           2696905|
|   mean|137053.96543856172|
| stddev| 6046342.246458627|
|    min|        $1,001,580|
|    max|   the Memorial/Mu|
+-------+------------------+



## 5. Generate summary statistics for TWO variables
Now try to generate ONLY the count min and max for BOTH the "Value" and "Season" variable using the select. You can't use the .describe function for this one but see if you can remember which function you CAN use. 

In [10]:
historical_dataframe.select("Value", "Season").summary("count", "min", "max").show()

+-------+---------------+-------+
|summary|          Value| Season|
+-------+---------------+-------+
|  count|        2696905|2740403|
|    min|     $1,001,580|   2010|
|    max|the Memorial/Mu|   2018|
+-------+---------------+-------+



## 6. Write a parquet file

Now try writing a parquet file (not partitioned) from the pga dataset. But first create a new dataframe containing ONLY the the "Season" and "Value" fields (using the "select command you used in the question above) and write a parquet file partitioned by "Season". This is a bit of a challenge aimed at getting you ready for material that will be covered later on in the course. Don't feel bad if you can't figure it out.

*Note that if any of your variable names contain spaces, spark will produce an error message with this call. That is why we are selecting ONLY the "Season" and "Value" fields. Ideally we should renamed those columns but we haven't gotten to that yet in this course but we will soon!*

In [123]:
values = historical_dataframe.select("Value", "Season").collect()
data_frame = spark.createDataFrame(values)
# Writing a parquete file 
data_frame.write.mode('overwrite').parquet('parquet_not_partitioned')


25/11/15 20:09:29 WARN TaskSetManager: Stage 13 contains a task of very large size (7605 KiB). The maximum recommended task size is 1000 KiB.


## 7. Write a partioned parquet file

You will need to use the same limited dataframe that you created in the previous question to accomplish this task as well. 

In [ ]:
# Writing a parquet file partitioned by Season
data_frame.write.mode('overwrite').partitionBy('Season').parquet("parquet_partitioned")

25/11/15 14:12:05 WARN TaskSetManager: Stage 12 contains a task of very large size (7605 KiB). The maximum recommended task size is 1000 KiB.


## 8. Read in a partitioned parquet file

Now try reading in the partitioned parquet file you just created above. 

In [ ]:
partitioned=spark.read.parquet('partitioned')
partitioned

DataFrame[Value: string, Season: int]

## 9. Reading in a set of paritioned parquet files

Now try only reading Seasons 2010, 2011 and 2012.

In [ ]:
#path="Database"
#users1_2 = spark.read.option("basePath", path).parquet(path+'users1.parquet', path+'users2.parquet')
parquet_files = partitioned.filter(partitioned.Season.isin(2010,2011,2012))
parquet_files.show(50)

+-----+------+
|Value|Season|
+-----+------+
|   71|  2010|
|   77|  2010|
|   83|  2010|
|   54|  2010|
|  100|  2010|
|   63|  2010|
|   88|  2010|
|   64|  2010|
|   64|  2010|
|   92|  2010|
|   75|  2010|
|   54|  2010|
|   76|  2010|
|   94|  2010|
|   82|  2010|
|   85|  2010|
|   79|  2010|
|   89|  2010|
|   88|  2010|
|   91|  2010|
|   91|  2010|
|   84|  2010|
|   54|  2010|
|   73|  2010|
|   74|  2010|
|   84|  2010|
|   75|  2010|
|   88|  2010|
|   53|  2010|
|   81|  2010|
|   87|  2010|
|   55|  2010|
|   70|  2010|
|  103|  2010|
|   64|  2010|
|   91|  2010|
|   81|  2010|
|   73|  2010|
|   78|  2010|
|   61|  2010|
|   79|  2010|
|   87|  2010|
|   89|  2010|
|  101|  2010|
|   82|  2010|
|   92|  2010|
|   98|  2010|
|   58|  2010|
|   84|  2010|
|   66|  2010|
+-----+------+
only showing top 50 rows


## 10. Create your own dataframe

Try creating your own dataframe below using PySparks *.createDataFrame* function. See if you can make one that contains 4 variables and at least 3 rows. 

Let's see how creative you can get on the content of the dataframe :)

In [11]:
data =[("Latifah", 25,"Rwanda","Engineer"),
       ("Winny", 29,"Kenya","Doctor"),
       ("Allen", 28,"Rwanda","Artist"),
       ("Ken", 35,"uganda","Designer"),
       ("Simon", 30,"Nigeria","Project Manager"),
]
df = spark.createDataFrame(data, ["Name", "Age", "Country", "Occupation"])
df.show()

+-------+---+-------+---------------+
|   Name|Age|Country|     Occupation|
+-------+---+-------+---------------+
|Latifah| 25| Rwanda|       Engineer|
|  Winny| 29|  Kenya|         Doctor|
|  Allen| 28| Rwanda|         Artist|
|    Ken| 35| uganda|       Designer|
|  Simon| 30|Nigeria|Project Manager|
+-------+---+-------+---------------+



In [125]:
spark.stop()

# **Part 2: Manipulating Data in DataFrames HW**


#### Let's get started applying what we learned in the lecure!

I've provided several questions below to help test and expand you knowledge from the code along lecture. So let's see what you've got!

First create your spark instance as we need to do at the start of every project.

In [126]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Part2").getOrCreate()
spark


## Read in our Republican vs. Democrats Tweet DataFrame

Attached to the lecture

In [127]:
repub_demo_df = spark.read.csv('Datasets/Rep_vs_Dem_tweets.csv', inferSchema=True, header=True )

## About this dataframe

Extracted tweets from all of the representatives (latest 200 as of May 17th 2018)

**Source:** https://www.kaggle.com/kapastor/democratvsrepublicantweets#ExtractedTweets.csv

Use either .show() or .toPandas() check out the first view rows of the dataframe to get an idea of what we are working with.

In [128]:
repub_demo_df.toPandas()

,Party,Handle,Tweet
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P..."
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...
3,"Congress has allocated about $18…""",None,None
4,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...
...,...,...,...
92484,Republican,RepTomPrice,Check out my op-ed on need for End Executive O...
92485,Republican,RepTomPrice,"Yesterday, Betty &amp; I had a great time lear..."
92486,Republican,RepTomPrice,We are forever grateful for the service and sa...
92487,Republican,RepTomPrice,Happy first day of school @CobbSchools! #CobbB...


In [129]:
repub_demo_df.show()

+--------------------+-------------+--------------------+
|               Party|       Handle|               Tweet|
+--------------------+-------------+--------------------+
|            Democrat|RepDarrenSoto|Today, Senate Dem...|
|            Democrat|RepDarrenSoto|RT @WinterHavenSu...|
|            Democrat|RepDarrenSoto|RT @NBCLatino: .@...|
|Congress has allo...|         NULL|                NULL|
|            Democrat|RepDarrenSoto|RT @NALCABPolicy:...|
|            Democrat|RepDarrenSoto|RT @Vegalteno: Hu...|
|            Democrat|RepDarrenSoto|RT @EmgageActionF...|
|            Democrat|RepDarrenSoto|Hurricane Maria l...|
|            Democrat|RepDarrenSoto|RT @Tharryry: I a...|
|            Democrat|RepDarrenSoto|RT @HispanicCaucu...|
|            Democrat|RepDarrenSoto|RT @RepStephMurph...|
|            Democrat|RepDarrenSoto|RT @AllSaints_FL:...|
|            Democrat|RepDarrenSoto|.@realDonaldTrump...|
|            Democrat|RepDarrenSoto|Thank you to my m...|
|            D

**Prevent Truncation of view**

If the view you produced above truncated some of the longer tweets, see if you can prevent that so you can read the whole tweet.

In [130]:
repub_demo_df.show(truncate=False)

+----------------------------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Party                                               |Handle       |Tweet                                                                                                                                       |
+----------------------------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Democrat                                            |RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |
|Democrat                                            |RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized 

**Print Schema**

First, check the schema to make sure the datatypes are accurate. 

In [131]:
repub_demo_df.printSchema()

root
 |-- Party: string (nullable = true)
 |-- Handle: string (nullable = true)
 |-- Tweet: string (nullable = true)



## 1. Can you identify any tweet that mentions the handle @LatinoLeader using regexp_extract?

It doesn't matter how you identify the row, any identifier will do. You can test your script on row 5 from this dataset. That row contains @LatinoLeader. 

In [132]:
from pyspark.sql.functions import regexp_extract
pattern = r"@LatinoLeader"
df_with_pattern = repub_demo_df.withColumn("Tweet", regexp_extract("Tweet", pattern, 0))
df_with_pattern.show(truncate=False)

+----------------------------------------------------+-------------+-------------+
|Party                                               |Handle       |Tweet        |
+----------------------------------------------------+-------------+-------------+
|Democrat                                            |RepDarrenSoto|             |
|Democrat                                            |RepDarrenSoto|             |
|Democrat                                            |RepDarrenSoto|             |
|Congress has allocated about $18…"                  |NULL         |NULL         |
|Democrat                                            |RepDarrenSoto|@LatinoLeader|
|Democrat                                            |RepDarrenSoto|             |
|Democrat                                            |RepDarrenSoto|             |
|Democrat                                            |RepDarrenSoto|             |
|Democrat                                            |RepDarrenSoto|             |
|Dem

## 2. Replace any value other than 'Democrate' or 'Republican' with 'Other' in the Party column.

We can see from the output below, that there are several other values other than 'Democrate' or 'Republican' in the Part column. We are assuming that this is dirty data that needs to be cleaned up.

In [135]:

from pyspark.sql.functions import when

repub_demo_df = repub_demo_df.withColumn("Party", when(repub_demo_df.Party=="Democrat", 'Democrat').when(repub_demo_df.Party == "Republican", "Republican").otherwise('Other'))
repub_demo_df.show(truncate=False)


+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Party   |Handle       |Tweet                                                                                                                                       |
+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Democrat|RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |
|Democrat|RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|
|Democrat|RepDarrenSoto|RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.                                    |
|Oth

## 3. Delete all embedded links (ie. "https:....)

For example see the first row in the tweets dataframe. 

*Note: this may require an google search :)*

In [136]:
from pyspark.sql.functions import regexp_replace
link_pattern = r"https?://\S+"
repub_demo_df = repub_demo_df.withColumn("Tweet", regexp_replace(repub_demo_df.Tweet, link_pattern, ""))
repub_demo_df.show(truncate=False)

+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Party   |Handle       |Tweet                                                                                                                                       |
+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Democrat|RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House…                         |
|Democrat|RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|
|Democrat|RepDarrenSoto|RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.                                    |
|Oth

## 4. Remove any leading or trailing white space in the tweet column

In [137]:
from pyspark.sql.functions import trim
repub_demo_df = repub_demo_df.withColumn('Tweet', trim(repub_demo_df.Tweet))
repub_demo_df.show(50, truncate=False)


+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Party   |Handle       |Tweet                                                                                                                                       |
+--------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Democrat|RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House…                         |
|Democrat|RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|
|Democrat|RepDarrenSoto|RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.                                    |
|Oth

## 5. Rename the 'Party' column to 'Dem_Rep'

No real reason here :) just wanted you to get practice doing this. 

In [138]:
rename_column = repub_demo_df.withColumnRenamed('Party', 'Dem_Rep')
rename_column.show(5)

+--------+-------------+--------------------+
| Dem_Rep|       Handle|               Tweet|
+--------+-------------+--------------------+
|Democrat|RepDarrenSoto|Today, Senate Dem...|
|Democrat|RepDarrenSoto|RT @WinterHavenSu...|
|Democrat|RepDarrenSoto|RT @NBCLatino: .@...|
|   Other|         NULL|                NULL|
|Democrat|RepDarrenSoto|RT @NALCABPolicy:...|
+--------+-------------+--------------------+
only showing top 5 rows


## 6. Concatenate the Party and Handle columns

Silly yes... but good practice.

pyspark.sql.functions.concat_ws(sep, *cols)[source] <br>
Concatenates multiple input string columns together into a single string column, using the given separator.

In [140]:
concatenated_df = repub_demo_df.select(repub_demo_df.Party, repub_demo_df.Handle, concat_ws(' ', repub_demo_df.Party, repub_demo_df.Handle).alias("Party_Handle")).show(6, truncate=False)

+--------+-------------+----------------------+
|Party   |Handle       |Party_Handle          |
+--------+-------------+----------------------+
|Democrat|RepDarrenSoto|Democrat RepDarrenSoto|
|Democrat|RepDarrenSoto|Democrat RepDarrenSoto|
|Democrat|RepDarrenSoto|Democrat RepDarrenSoto|
|Other   |NULL         |Other                 |
|Democrat|RepDarrenSoto|Democrat RepDarrenSoto|
|Democrat|RepDarrenSoto|Democrat RepDarrenSoto|
+--------+-------------+----------------------+
only showing top 6 rows


## Challenge Question

Let's image that we want to analyze the hashtags that are used in these tweets. Can you extract all the hashtags you see?

In [141]:
from pyspark.sql.functions import regexp_extract
pattern = r"#\w+"
df_hastags = repub_demo_df.withColumn("Tweet", regexp_extract("Tweet", pattern, 0))
df_hastags.show(60, truncate=False)

+--------+-------------+------------------------+
|Party   |Handle       |Tweet                   |
+--------+-------------+------------------------+
|Democrat|RepDarrenSoto|#SaveTheInternet        |
|Democrat|RepDarrenSoto|                        |
|Democrat|RepDarrenSoto|                        |
|Other   |NULL         |NULL                    |
|Democrat|RepDarrenSoto|#NALCABPolicy2018       |
|Democrat|RepDarrenSoto|                        |
|Democrat|RepDarrenSoto|                        |
|Democrat|RepDarrenSoto|                        |
|Democrat|RepDarrenSoto|#NetNeutrality          |
|Democrat|RepDarrenSoto|                        |
|Democrat|RepDarrenSoto|#Orlando                |
|Democrat|RepDarrenSoto|                        |
|Democrat|RepDarrenSoto|                        |
|Democrat|RepDarrenSoto|                        |
|Democrat|RepDarrenSoto|                        |
|Other   |NULL         |NULL                    |
|Democrat|RepDarrenSoto|                        |


# Let's create our own dataset to work with real dates

This is a dataset of patient visits from a medical office. It contains the patients first and last names, date of birth, and the dates of their first 3 visits. 

In [153]:
from pyspark.sql.types import *

md_office = [('Mohammed','Alfasy','1987-4-8','2016-1-7','2017-2-3','2018-3-2') \
            ,('Marcy','Wellmaker','1986-4-8','2015-1-7','2017-1-3','2018-1-2') \
            ,('Ginny','Ginger','1986-7-10','2014-8-7','2015-2-3','2016-3-2') \
            ,('Vijay','Doberson','1988-5-2','2016-1-7','2018-2-3','2018-3-2') \
            ,('Orhan','Gelicek','1987-5-11','2016-5-7','2017-1-3','2018-9-2') \
            ,('Sarah','Jones','1956-7-6','2016-4-7','2017-8-3','2018-10-2') \
            ,('John','Johnson','2017-10-12','2018-1-2','2018-10-3','2018-3-2') ]

df = spark.createDataFrame(md_office,['first_name','last_name','dob','visit1','visit2','visit3']) # schema=final_struc

# Check to make sure it worked
df.show()
print(df.printSchema())

+----------+---------+----------+--------+---------+---------+
|first_name|last_name|       dob|  visit1|   visit2|   visit3|
+----------+---------+----------+--------+---------+---------+
|  Mohammed|   Alfasy|  1987-4-8|2016-1-7| 2017-2-3| 2018-3-2|
|     Marcy|Wellmaker|  1986-4-8|2015-1-7| 2017-1-3| 2018-1-2|
|     Ginny|   Ginger| 1986-7-10|2014-8-7| 2015-2-3| 2016-3-2|
|     Vijay| Doberson|  1988-5-2|2016-1-7| 2018-2-3| 2018-3-2|
|     Orhan|  Gelicek| 1987-5-11|2016-5-7| 2017-1-3| 2018-9-2|
|     Sarah|    Jones|  1956-7-6|2016-4-7| 2017-8-3|2018-10-2|
|      John|  Johnson|2017-10-12|2018-1-2|2018-10-3| 2018-3-2|
+----------+---------+----------+--------+---------+---------+

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- visit1: string (nullable = true)
 |-- visit2: string (nullable = true)
 |-- visit3: string (nullable = true)

None


Oh no! The dates are still stored as text... let's try converting them again and see if we have any issues this time.

In [143]:

df = df.withColumn("dob", df["dob"].cast(DateType()))\
.withColumn("visit1", df['visit1'].cast(DateType()))\
.withColumn("visit2", df['visit2'].cast(DateType()))\
.withColumn("visit3", df['visit3'].cast(DateType()))

df.printSchema()


root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- visit1: date (nullable = true)
 |-- visit2: date (nullable = true)
 |-- visit3: date (nullable = true)



## 7. Can you calculate a variable showing the length of time between patient visits?

Compare visit1 to visit2 and visit2 to visit3 for all patients and see what the average length of time is between visits. Create an alias for it as well. 

In [144]:
visit_2_1 = df.select(datediff(df.visit2, df.visit1).alias('visit_2_1')).show()

+---------+
|visit_2_1|
+---------+
|      393|
|      727|
|      180|
|      758|
|      241|
|      483|
|      274|
+---------+



In [145]:
visit_3_2 = df.select(datediff(df.visit3, df.visit2).alias('visit_3_2')).show()

+---------+
|visit_3_2|
+---------+
|      392|
|      364|
|      393|
|       27|
|      607|
|      425|
|     -215|
+---------+



## 8. Can you calculate the age of each patient?

In [158]:
import pyspark.sql.functions as fn
age = fn.year(fn.current_date()) - fn.year('dob')
df.withColumn("age", age).show()

+----------+---------+----------+--------+---------+---------+---+
|first_name|last_name|       dob|  visit1|   visit2|   visit3|age|
+----------+---------+----------+--------+---------+---------+---+
|  Mohammed|   Alfasy|  1987-4-8|2016-1-7| 2017-2-3| 2018-3-2| 38|
|     Marcy|Wellmaker|  1986-4-8|2015-1-7| 2017-1-3| 2018-1-2| 39|
|     Ginny|   Ginger| 1986-7-10|2014-8-7| 2015-2-3| 2016-3-2| 39|
|     Vijay| Doberson|  1988-5-2|2016-1-7| 2018-2-3| 2018-3-2| 37|
|     Orhan|  Gelicek| 1987-5-11|2016-5-7| 2017-1-3| 2018-9-2| 38|
|     Sarah|    Jones|  1956-7-6|2016-4-7| 2017-8-3|2018-10-2| 69|
|      John|  Johnson|2017-10-12|2018-1-2|2018-10-3| 2018-3-2|  8|
+----------+---------+----------+--------+---------+---------+---+



## 9. Can you extract the month from the first visit column and call it "Month"?

In [159]:
month = df.withColumn("month", fn.month('visit1')).show()


+----------+---------+----------+--------+---------+---------+-----+
|first_name|last_name|       dob|  visit1|   visit2|   visit3|month|
+----------+---------+----------+--------+---------+---------+-----+
|  Mohammed|   Alfasy|  1987-4-8|2016-1-7| 2017-2-3| 2018-3-2|    1|
|     Marcy|Wellmaker|  1986-4-8|2015-1-7| 2017-1-3| 2018-1-2|    1|
|     Ginny|   Ginger| 1986-7-10|2014-8-7| 2015-2-3| 2016-3-2|    8|
|     Vijay| Doberson|  1988-5-2|2016-1-7| 2018-2-3| 2018-3-2|    1|
|     Orhan|  Gelicek| 1987-5-11|2016-5-7| 2017-1-3| 2018-9-2|    5|
|     Sarah|    Jones|  1956-7-6|2016-4-7| 2017-8-3|2018-10-2|    4|
|      John|  Johnson|2017-10-12|2018-1-2|2018-10-3| 2018-3-2|    1|
+----------+---------+----------+--------+---------+---------+-----+



## 10. Challenges with working with date and timestamps

Let's read in the supermarket sales dataframe attached to the lecture now and see some of the issues that can come up when working with date and timestamps values.

In [160]:
supermarket_sales = spark.read.csv('Datasets/supermarket_sales.csv', inferSchema=True, header = True)

## About this dataset

The growth of supermarkets in most populated cities are increasing and market competitions are also high. The dataset is one of the historical sales of supermarket company which has recorded in 3 different branches for 3 months data. 

 - Attribute information
 - Invoice id: Computer generated sales slip invoice identification number
 - Branch: Branch of supercenter (3 branches are available identified by A, B and C).
 - City: Location of supercenters
 - Customer type: Type of customers, recorded by Members for customers using member card and Normal for without member card.
 - Gender: Gender type of customer
 - Product line: General item categorization groups - Electronic accessories, Fashion accessories, Food and beverages, Health and beauty, Home and lifestyle, Sports and travel
 - Unit price: Price of each product in USD
 - Quantity: Number of products purchased by customer
 - Tax: 5% tax fee for customer buying
 - Total: Total price including tax
 - Date: Date of purchase (Record available from January 2019 to March 2019)
 - Time: Purchase time (10am to 9pm)
 - Payment: Payment used by customer for purchase (3 methods are available – Cash, Credit card and Ewallet)
 - COGS: Cost of goods sold
 - Gross margin percentage: Gross margin percentage
 - Gross income: Gross income
 - Rating: Customer stratification rating on their overall shopping experience (On a scale of 1 to 10)

**Source:** https://www.kaggle.com/aungpyaeap/supermarket-sales

### View dataframe and schema as usual

In [161]:
supermarket_sales.show()

+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+---------+-------------------+-----------+------+-----------------------+------------+------+
| Invoice ID|Branch|     City|Customer type|Gender|        Product line|Unit price|Quantity| Tax 5%|   Total|     Date|               Time|    Payment|  cogs|gross margin percentage|gross income|Rating|
+-----------+------+---------+-------------+------+--------------------+----------+--------+-------+--------+---------+-------------------+-----------+------+-----------------------+------------+------+
|750-67-8428|     A|   Yangon|       Member|Female|   Health and beauty|     74.69|       7|26.1415|548.9715| 1/5/2019|2025-11-15 13:08:00|    Ewallet|522.83|            4.761904762|     26.1415|   9.1|
|226-31-3081|     C|Naypyitaw|       Normal|Female|Electronic access...|     15.28|       5|   3.82|   80.22| 3/8/2019|2025-11-15 10:29:00|       Cash|  76.4|            4.761904762|      

In [162]:
supermarket_sales.printSchema()

root
 |-- Invoice ID: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Customer type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Unit price: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Tax 5%: double (nullable = true)
 |-- Total: double (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: timestamp (nullable = true)
 |-- Payment: string (nullable = true)
 |-- cogs: double (nullable = true)
 |-- gross margin percentage: double (nullable = true)
 |-- gross income: double (nullable = true)
 |-- Rating: double (nullable = true)



### Convert date field to date type

Looks like we need to convert the date field into a date type. Let's go ahead and do that..

In [163]:
from pyspark.sql.functions import to_date

supermarket_sales = supermarket_sales.withColumn(
    "Date",
    to_date("Date", "M/d/yyyy")
)

In [164]:
supermarket_sales.schema['Date'].dataType

DateType()

### How can we extract the month value from the date field?

If you had trouble converting the date field in the previous question think about a more creative solution to extract the month from that field.

In [167]:
from pyspark.sql.functions import month, col
supermarket_sales = supermarket_sales.withColumn("month", month(col('Date')))
supermarket_sales.select("Date", "month").show()

+----------+-----+
|      Date|month|
+----------+-----+
|2019-01-05|    1|
|2019-03-08|    3|
|2019-03-03|    3|
|2019-01-27|    1|
|2019-02-08|    2|
|2019-03-25|    3|
|2019-02-25|    2|
|2019-02-24|    2|
|2019-01-10|    1|
|2019-02-20|    2|
|2019-02-06|    2|
|2019-03-09|    3|
|2019-02-12|    2|
|2019-02-07|    2|
|2019-03-29|    3|
|2019-01-15|    1|
|2019-03-11|    3|
|2019-01-01|    1|
|2019-01-21|    1|
|2019-03-11|    3|
+----------+-----+
only showing top 20 rows


## 11.0 Working with Arrays

Here is a dataframe of reviews from the movie the Dark Night.

In [168]:
from pyspark.sql.functions import *

values = [(5,'Epic. This is the best movie I have EVER seen'), \
          (4,'Pretty good, but I would have liked to seen better special effects'), \
          (3,'So so. Casting could have been improved'), \
          (5,'The most EPIC movie of the year! Casting was awesome. Special effects were so intense.'), \
          (4,'Solid but I would have liked to see more of the love story'), \
          (5,'THE BOMB!!!!!!!')]
reviews = spark.createDataFrame(values,['rating', 'review_txt'])

reviews.show(6,False)

+------+--------------------------------------------------------------------------------------+
|rating|review_txt                                                                            |
+------+--------------------------------------------------------------------------------------+
|5     |Epic. This is the best movie I have EVER seen                                         |
|4     |Pretty good, but I would have liked to seen better special effects                    |
|3     |So so. Casting could have been improved                                               |
|5     |The most EPIC movie of the year! Casting was awesome. Special effects were so intense.|
|4     |Solid but I would have liked to see more of the love story                            |
|5     |THE BOMB!!!!!!!                                                                       |
+------+--------------------------------------------------------------------------------------+



## 11.1 Let's see if we can create an array off of the review text column and then derive some meaningful results from it.

**But first** we need to clean the rview_txt column to make sure we can get what we need from our analysis later on. So let's do the following:

1. Remove all punctuation
2. lower case everything
3. Remove white space (trim)
3. Then finally, split the string

In [169]:
# Remove all punctuation
from pyspark.sql.functions import regexp_replace
reviews_cleaned = reviews.withColumn("review_txt", regexp_replace(reviews.review_txt, "[^a-zA-Z\\s]", ""))
reviews_cleaned.show(truncate=False)

+------+-----------------------------------------------------------------------------------+
|rating|review_txt                                                                         |
+------+-----------------------------------------------------------------------------------+
|5     |Epic This is the best movie I have EVER seen                                       |
|4     |Pretty good but I would have liked to seen better special effects                  |
|3     |So so Casting could have been improved                                             |
|5     |The most EPIC movie of the year Casting was awesome Special effects were so intense|
|4     |Solid but I would have liked to see more of the love story                         |
|5     |THE BOMB                                                                           |
+------+-----------------------------------------------------------------------------------+



In [170]:
# lower case everything
reviews_cleaned = reviews_cleaned.withColumn("review_txt", lower(reviews_cleaned.review_txt))
reviews_cleaned.show(truncate=False)

+------+-----------------------------------------------------------------------------------+
|rating|review_txt                                                                         |
+------+-----------------------------------------------------------------------------------+
|5     |epic this is the best movie i have ever seen                                       |
|4     |pretty good but i would have liked to seen better special effects                  |
|3     |so so casting could have been improved                                             |
|5     |the most epic movie of the year casting was awesome special effects were so intense|
|4     |solid but i would have liked to see more of the love story                         |
|5     |the bomb                                                                           |
+------+-----------------------------------------------------------------------------------+



In [171]:
# Remove white space (trim)
reviews_cleaned =reviews_cleaned.withColumn('review_txt', trim(reviews_cleaned.review_txt))
reviews_cleaned.show(truncate=False)

+------+-----------------------------------------------------------------------------------+
|rating|review_txt                                                                         |
+------+-----------------------------------------------------------------------------------+
|5     |epic this is the best movie i have ever seen                                       |
|4     |pretty good but i would have liked to seen better special effects                  |
|3     |so so casting could have been improved                                             |
|5     |the most epic movie of the year casting was awesome special effects were so intense|
|4     |solid but i would have liked to see more of the love story                         |
|5     |the bomb                                                                           |
+------+-----------------------------------------------------------------------------------+



In [172]:
# Split the string
reviews_cleaned = reviews_cleaned.withColumn("array", split(reviews_cleaned.review_txt, " "))
reviews_cleaned.show()

+------+--------------------+--------------------+
|rating|          review_txt|               array|
+------+--------------------+--------------------+
|     5|epic this is the ...|[epic, this, is, ...|
|     4|pretty good but i...|[pretty, good, bu...|
|     3|so so casting cou...|[so, so, casting,...|
|     5|the most epic mov...|[the, most, epic,...|
|     4|solid but i would...|[solid, but, i, w...|
|     5|            the bomb|         [the, bomb]|
+------+--------------------+--------------------+



## 11.2 Alright now let's see if we can find which reviews contain the word 'Epic'

In [173]:
reviews.filter(lower(reviews.review_txt).contains("epic")).show(truncate=False)

+------+--------------------------------------------------------------------------------------+
|rating|review_txt                                                                            |
+------+--------------------------------------------------------------------------------------+
|5     |Epic. This is the best movie I have EVER seen                                         |
|5     |The most EPIC movie of the year! Casting was awesome. Special effects were so intense.|
+------+--------------------------------------------------------------------------------------+



In [174]:
spark.stop()

# **Part 3: Joining and Appending DataFrames in PySpark HW**

Now it's time to test your knowledge and further engrain the concepts we touched on in the lectures. Let's go ahead and get started.




**As always let's start our Spark instance.**

In [2]:

import findspark
findspark.init()

import pyspark 
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Part3").getOrCreate()
spark


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/15 20:47:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Read in the database

Let cotinue working with our college courses dataframe to get some more insights and practice what we have learned!Let's read in the whole database using the loop function that we learned about in the lecture to automatically read in all the datasets from the uw-madision-courses folder (there are too many datasets to each one individually.

In [3]:
import os

path = "madison_courses/"

df_list = []
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        filename_list = filename.split(".") 
        df_name = filename_list[0]
        df = spark.read.csv(path+filename,inferSchema=True,header=True)
        df.name = df_name
        df_list.append(df_name)
        exec(df_name + ' = df')
        
print("Full list of data frames found:")
print(df_list)

25/11/15 20:48:05 WARN DAGScheduler: Broadcasting large task binary with size 8.2 MiB
25/11/15 20:48:08 WARN DAGScheduler: Broadcasting large task binary with size 28.1 MiB


Full list of data frames found:
['subjects', 'subject_memberships', 'rooms', 'schedules', 'sections', 'courses', 'course_offerings', 'instructors', 'teachings', 'grade_distributions']


Now check the contents of a few of the dataframses that were read in above.

In [4]:
grade_distributions.printSchema()

root
 |-- course_offering_uuid: string (nullable = true)
 |-- section_number: integer (nullable = true)
 |-- a_count: integer (nullable = true)
 |-- ab_count: integer (nullable = true)
 |-- b_count: integer (nullable = true)
 |-- bc_count: integer (nullable = true)
 |-- c_count: integer (nullable = true)
 |-- d_count: integer (nullable = true)
 |-- f_count: integer (nullable = true)
 |-- s_count: integer (nullable = true)
 |-- u_count: integer (nullable = true)
 |-- cr_count: integer (nullable = true)
 |-- n_count: integer (nullable = true)
 |-- p_count: integer (nullable = true)
 |-- i_count: integer (nullable = true)
 |-- nw_count: integer (nullable = true)
 |-- nr_count: integer (nullable = true)
 |-- other_count: integer (nullable = true)



In [5]:
grade_distributions.limit(15).toPandas()

,course_offering_uuid,section_number,a_count,ab_count,b_count,bc_count,c_count,d_count,f_count,s_count,u_count,cr_count,n_count,p_count,i_count,nw_count,nr_count,other_count
0,344b3ebe-da7e-314c-83ed-9425269695fd,1,105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,1,158,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,ea3b717c-d66b-30dc-8b37-964d9688295f,1,139,12,2,0,3,0,0,0,0,0,0,0,0,0,0,0
3,075da420-5f49-3dd0-93df-13e3c152e1b1,1,87,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,2b4e216d-a728-3713-8c7c-19afffc6b2fd,1,70,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
5,87966a7b-f676-33d0-83d2-acdb67da6790,1,79,0,0,0,0,0,0,0,0,0,0,0,12,0,0,0
6,ff511882-5eab-3e7b-a89f-8fbfd1906127,1,114,3,1,0,0,0,0,0,0,0,0,0,3,0,0,0
7,b3e26604-fe6e-30df-bc91-b5fa57717a7e,1,105,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
8,e100d196-5e82-32e4-80e9-ac45c07a498c,1,107,3,1,0,0,0,0,0,0,0,0,0,2,0,0,0
9,76b9c458-d3c2-38c4-951f-69b6900bd7fe,1,129,8,2,0,1,1,0,0,0,0,0,0,3,0,0,0


In [6]:
courses.printSchema()

root
 |-- uuid: string (nullable = true)
 |-- name: string (nullable = true)
 |-- number: integer (nullable = true)



In [7]:
courses.limit(15).toPandas()

,uuid,name,number
0,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,Cooperative Education Program,1
1,c070a84f-648f-351d-9499-5d0e30ad02cc,Cooperative Education/Co-op in Life Sciences C...,1
2,e6b4b7ae-0e0b-3aa5-9d77-7fcd90c9cfa3,Cooperative Education Program,1
3,8f63bde1-ff7f-3fe7-9901-862908bf134c,Workshop in Dance Activity,1
4,f3541888-584a-3923-9ce7-6341ff3d84a1,Cooperative Education/Co-op in Agricultural & ...,1
5,d07193c1-551e-3c4d-82cd-7059fc2cd512,Ballroom Dance I,2
6,eb1a8a08-a675-3c45-aa49-23a6589fed56,Fundamentals-Flute,7
7,57540351-0e4a-3528-9ac0-84e8f72b41bc,Contemporary Dance I,11
8,0422e04f-96d2-3b9f-ba5e-4ed3a43173f6,Fundamentals: Single Reeds,14
9,3e2fe18a-32f9-3e6f-a627-f77481eb84d9,Fundamentals-Low Brass,24


In [8]:
course_offerings.limit(15).toPandas()

,uuid,course_uuid,term_code,name
0,344b3ebe-da7e-314c-83ed-9425269695fd,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1092,Cooperative Education Prog
1,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1082,Cooperative Education Prog
2,ea3b717c-d66b-30dc-8b37-964d9688295f,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1172,Cooperative Education Prog
3,075da420-5f49-3dd0-93df-13e3c152e1b1,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1114,Cooperative Education Prog
4,2b4e216d-a728-3713-8c7c-19afffc6b2fd,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1104,Cooperative Education Prog
5,87966a7b-f676-33d0-83d2-acdb67da6790,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1112,Cooperative Education Prog
6,ff511882-5eab-3e7b-a89f-8fbfd1906127,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1134,Cooperative Education Prog
7,b3e26604-fe6e-30df-bc91-b5fa57717a7e,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1084,Cooperative Education Prog
8,e100d196-5e82-32e4-80e9-ac45c07a498c,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1154,Cooperative Education Prog
9,76b9c458-d3c2-38c4-951f-69b6900bd7fe,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1162,Cooperative Education Prog


In [9]:
rooms.printSchema()

root
 |-- uuid: string (nullable = true)
 |-- facility_code: string (nullable = true)
 |-- room_code: string (nullable = true)



In [10]:
rooms.limit(15).toPandas()

,uuid,facility_code,room_code
0,04368a56-c959-3e4b-8b3d-f4cc3538fea5,OFF CAMPUS,null
1,2cc50da3-ef0e-3572-a557-ca44930a0688,0032,0249
2,ebbf62b4-2ac3-356b-b0fa-7897f4446a17,0032,B101
3,ed828265-475b-31b4-b9a8-daec2a600449,0032,0549
4,b277dc8e-bab1-3a12-bc17-48d4a364d297,0032,0349
5,bab1dd92-c641-3d28-abd5-50c9573c480b,0032,0510
6,9759cb5f-a7d3-3d0c-b8b3-03910bd0153e,0469,2441
7,6af80b0b-b3e3-370a-925f-829869f007bc,0469,4411
8,50322d30-dd8f-3c65-9a75-78c5ff29d62c,0469,2511
9,b7211db3-303f-3605-bead-237e1f663932,0469,1341


In [11]:
course_offerings.printSchema()

root
 |-- uuid: string (nullable = true)
 |-- course_uuid: string (nullable = true)
 |-- term_code: integer (nullable = true)
 |-- name: string (nullable = true)



In [13]:
course_offerings.limit(15).toPandas()

,uuid,course_uuid,term_code,name
0,344b3ebe-da7e-314c-83ed-9425269695fd,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1092,Cooperative Education Prog
1,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1082,Cooperative Education Prog
2,ea3b717c-d66b-30dc-8b37-964d9688295f,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1172,Cooperative Education Prog
3,075da420-5f49-3dd0-93df-13e3c152e1b1,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1114,Cooperative Education Prog
4,2b4e216d-a728-3713-8c7c-19afffc6b2fd,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1104,Cooperative Education Prog
5,87966a7b-f676-33d0-83d2-acdb67da6790,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1112,Cooperative Education Prog
6,ff511882-5eab-3e7b-a89f-8fbfd1906127,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1134,Cooperative Education Prog
7,b3e26604-fe6e-30df-bc91-b5fa57717a7e,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1084,Cooperative Education Prog
8,e100d196-5e82-32e4-80e9-ac45c07a498c,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1154,Cooperative Education Prog
9,76b9c458-d3c2-38c4-951f-69b6900bd7fe,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1162,Cooperative Education Prog


## Recap: About this database

You will notice that there are several more tables in the uw-madision-courses folder than there are read in above. This so that you will have a chance to practice your own custom joins and learn about the relationships between a real database work. Sometimes we don't know how they are related and we need to figure it out! I'll save that for the HW :) 

Here is a look at some of the important variables we can use to join our tables:

 - course_offerings: uuid, course_uuid, term_code, name
 - instructors: id, name
 - schedules: uuid
 - sections: uuid, course_offering_uuid,room_uuid, schedule_uuid
 - teachings: instructor_id, section_uuid
 - courses: uuid
 - grade_distributions: course_offering_uuid,section_number
 - rooms: uuid, facility_code, room_code
 - subjects: code
 - subject_memberships: subject_code, course_offering_uuid
 
 **Source:** https://www.kaggle.com/Madgrades/uw-madison-courses
 
So alright, let's use this information to discover some insights from this data!

## 1a. Can you assign the room numbers to each section of each course?

Show only the rooms uuid, facility code, room number, term code and the name of the course from the course_offerings table.

In [14]:
first_join = sections.join(course_offerings, sections.course_offering_uuid == course_offerings.uuid, how="inner").select(['term_code', 'name', 'room_uuid'])
first_join.show()

+---------+--------------------+--------------------+
|term_code|                name|           room_uuid|
+---------+--------------------+--------------------+
|     1092|Cooperative Educa...|                null|
|     1082|Cooperative Educa...|                null|
|     1172|Cooperative Educa...|04368a56-c959-3e4...|
|     1172|Cooperative Educa...|                null|
|     1172|Cooperative Educa...|                null|
|     1172|Cooperative Educa...|04368a56-c959-3e4...|
|     1114|Cooperative Educa...|04368a56-c959-3e4...|
|     1114|Cooperative Educa...|                null|
|     1104|Cooperative Educa...|                null|
|     1104|Cooperative Educa...|04368a56-c959-3e4...|
|     1112|Cooperative Educa...|                null|
|     1134|Cooperative Educa...|04368a56-c959-3e4...|
|     1134|Cooperative Educa...|04368a56-c959-3e4...|
|     1134|Cooperative Educa...|                null|
|     1084|Cooperative Educa...|                null|
|     1084|Cooperative Educa

In [15]:
second_join = first_join.join(rooms, first_join.room_uuid == rooms.uuid, how="inner").select(['room_uuid', 'facility_code','room_code','term_code', 'name',]).dropDuplicates()
second_join.show(30,truncate=False)

+------------------------------------+-------------+---------+---------+------------------------------+
|room_uuid                           |facility_code|room_code|term_code|name                          |
+------------------------------------+-------------+---------+---------+------------------------------+
|85d48cdb-8a44-35ec-a5b6-fdd920aef550|0545         |4004     |1094     |Intro to Speech Composition   |
|eda7ac38-2d95-3114-b675-c49966779d6d|0545         |4046     |1102     |Intro to Speech Composition   |
|026d4b38-9cc3-361d-af8d-ef162119f51e|0140         |2170     |1162     |Intro Financial Accounting    |
|913b30ca-3e8f-393b-bec9-6683abc6beec|0046         |5106     |1084     |Intro to Cultures of Asia     |
|babe81ae-64cc-3b30-b6ee-cb0f21c79c88|0070         |0125     |1094     |General Microbiology          |
|34d2c6bf-a685-3154-810c-f35db51bd417|0469         |3650     |1172     |Principles-Microeconomics     |
|5acfb8fa-8860-34bf-b551-52c7730f0a7e|0046         |6314     |11

## 1b. Now show same output as above but for only facility number 0469 (facility_code)

In [16]:
only_facility = second_join.filter(second_join.facility_code == "0469")
only_facility.show(30, truncate=False)

+------------------------------------+-------------+---------+---------+------------------------------+
|room_uuid                           |facility_code|room_code|term_code|name                          |
+------------------------------------+-------------+---------+---------+------------------------------+
|34d2c6bf-a685-3154-810c-f35db51bd417|0469         |3650     |1172     |Principles-Microeconomics     |
|e9f73325-cab9-3cf9-a2fe-7c954a0c80b2|0469         |2451     |1112     |The Musical Experience        |
|fce290a4-ed7d-30cf-ab0a-e4c64ecfce09|0469         |2125     |1094     |Amer Hist to Civil War Era    |
|d90fd659-4ffa-3536-87b1-381855199a0f|0469         |2611     |1092     |Amer Hist-Civil War-Present   |
|d90fd659-4ffa-3536-87b1-381855199a0f|0469         |2611     |1104     |The Symphony                  |
|00593bf2-b562-3482-9bb3-e2b5e7e00a16|0469         |1101     |1082     |Foundations of Contemp Art    |
|2da69d83-787a-3b03-a82b-133026765ef1|0469         |2251     |11

## 2. Count how many sections are offered for each subject for each facility

*Note: this will involve a groupby*

In [17]:
second_join.groupBy('facility_code').count().show()

+-------------+-----+
|facility_code|count|
+-------------+-----+
|         0469| 2671|
|         0452|    4|
|         0047|  863|
|         0407|  146|
|         0545|  988|
|         0153|   16|
|         0031|   72|
|         0140|  535|
|         0430|    3|
|         0080|   15|
|         0021|   13|
|         0060|   50|
|         0032|  213|
|         0576|    2|
|         0082|   22|
|         0520|   23|
|         0503|    4|
|         0084|   52|
|         0055|  566|
|         0482| 2644|
+-------------+-----+
only showing top 20 rows


## 3. What are the hardest classes?

Let's see if we can figure out which classes are the hardest by seeing how many students failed. Note that you will first need to aggregate the grades table by the course uuid to include all sections. Show the name of the course as well that you will need to get from the course_offering table.

In [19]:
from pyspark.sql.functions import col, sum, desc

course_failures = grade_distributions.groupBy('course_offering_uuid').agg(

    sum(col('f_count')).alias('total_failures'))

hardest_classes = course_failures.join(
    course_offerings,
    course_failures.course_offering_uuid == course_offerings.uuid,
    how='inner').select(course_offerings['name'].alias('course_name'),
    col('total_failures'))

hardest_classes.orderBy(desc('total_failures')).show(30, truncate=False)


+----------------------------+--------------+
|course_name                 |total_failures|
+----------------------------+--------------+
|Calc--Functns of Variables  |72            |
|Animal Biology              |70            |
|Calculus&Analytic Geometry  |67            |
|Calculus&Analytic Geometry 1|64            |
|Calculus&Analytic Geometry  |63            |
|Calculus&Analytic Geometry  |59            |
|Calculus&Analytic Geometry  |58            |
|Calculus&Analytic Geometry 1|57            |
|Animal Biology              |56            |
|Animal Biology              |54            |
|Animal Biology              |53            |
|Calculus&Analytic Geometry  |53            |
|Animal Biology              |52            |
|Calculus&Analytic Geometry  |52            |
|Intro Organic Chemistry     |52            |
|Intro Organic Chemistry     |51            |
|Calculus                    |50            |
|Calculus&Analytic Geometry  |49            |
|Intro Organic Chemistry     |49  

## Challenge Question: Automating data entry errors

We see in the dataframe below that there are several typos of various animal names. If this was a large database of several millions of records, correcting these errors would be way too labor intensive. How can we automate correcting these errors?

*Hint: Leven...*

In [192]:
values = [('Monkey',10),('Monkay',36),('Mnky',123), \
          ('Elephant',48),('Elefant',16),('Ellafant',1), \
          ('Hippopotamus',48),('Hipopotamus',16),('Hippo',1)]
zoo = spark.createDataFrame(values,['Animal','age'])
zoo.show()

+------------+---+
|      Animal|age|
+------------+---+
|      Monkey| 10|
|      Monkay| 36|
|        Mnky|123|
|    Elephant| 48|
|     Elefant| 16|
|    Ellafant|  1|
|Hippopotamus| 48|
| Hipopotamus| 16|
|       Hippo|  1|
+------------+---+



In [21]:
spark.stop()